In [7]:
from keras.layers import Dense, Dropout
from keras import Sequential
from tqdm import tqdm_notebook as tqdm
import pandas as pd
import re

from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import wordnet
from nltk.stem import SnowballStemmer

import numpy as np
import os
import string 
import re 
from nltk.tokenize import MWETokenizer, word_tokenize
from tqdm import tqdm
import spacy
from gensim.models import KeyedVectors
from main import *

SyntaxError: non-default argument follows default argument (main.py, line 21)

In [ ]:
def tokenization(corpus, tool, language = 'en', pos_to_remove = ['PUNCT','NUM'], ent_to_remove = ['PERSON','ORG'], stop_words_to_remove= False, lowercase = True, regex=True):
    """
    tool: one of two strings - 'spacy' or 'NLTK'
    languages (string ISO code): supports 'en', 'fi' or 'zh'
    pos_to_remove (list): part-of-speech tag from spacy
    ent_to_remove (list): entities from spacy
    
    
    """
    tokenized_corpus = []
    if tool == 'spacy':
        if language == 'en':
            sc = spacy.load('en_core_web_sm')
            for doc in tqdm(sc.pipe(corpus, disable=["lemmatizer", "textcat", "custom"])):
                if stop_words_to_remove:
                    doc_list = [word.text for word in doc if word.pos_ not in pos_to_remove if word.ent_type_ not in ent_to_remove if word.is_stop]
                else: 
                    doc_list = [word.text for word in doc if word.pos_ not in pos_to_remove if word.ent_type_ not in ent_to_remove]

                if lowercase:
                    doc_list = [word.lower() for word in doc_list]

                tokenized_corpus.append(doc_list)
        if language == 'zh':
            sc = spacy.load('zh_core_web_sm')
            for doc in tqdm(sc.pipe(corpus, disable=["lemmatizer", "textcat", "custom"])):
                if stop_words_to_remove:
                    doc_list = [word.text for word in doc if word.pos_ not in ['PUNCT'] if word.is_stop]
                else: 
                    doc_list = [word.text for word in doc if word.pos_ not in ['PUNCT']]
                
                tokenized_corpus.append(doc_list)
        if language == 'fi':
            sc = spacy.load('xx_sent_ud_sm')
            for doc in tqdm(sc.pipe(corpus, disable=["lemmatizer", "textcat", "custom"])): 
                doc_list = [word.text for word in doc]
                
                if lowercase:
                    doc_list = [word.lower() for word in doc_list]

                tokenized_corpus.append(doc_list)
                
            
        
    if tool == 'NLTK':
        print('Not implemented')
        
        
    return tokenized_corpus


def match_regex(tokenized_corpus, letters = True, letters_and_numbers = False):
    
    if letters:
        regex = r'[a-z]+'
    if letters_and_numbers:
        regex = r'([a-z]+|^\d+$)'
        
    new_tokenized = []
    for sentence_list in tokenized_corpus:
        sentence_list2 = [word for word in sentence_list if re.search(regex, word)]
        new_tokenized.append(sentence_list2)
        
    return new_tokenized

def chinese_regex(tokenized_corpus, chinese = True, chinese_and_numbers = False):
    
    if chinese:
        regex = r'[\u4e00-\u9fff]+'
    if chinese_and_numbers:
        regex = r'([\u4e00-\u9fff]+|^\d+$)'
        
    new_tokenized_zh = []
    for sentence_list in tokenized_corpus:
        chinese = [word for word in sentence_list if re.search(regex, word)]
        new_tokenized_zh.append(chinese)
        
    return new_tokenized_zh


In [2]:
def the_preprocessor2(df,lang, tool="spacy", pos_list=[], ent_list=[], stopwords=False, lowercase=True, let=True, letandnum=False):
    if lang == "en":
        word_vect = KeyedVectors.load('en_vectors.kv')

        df["pt"] = tokenization(df["translation"], tool = tool, language = lang, pos_to_remove = pos_list, ent_to_remove = ent_list, stop_words_to_remove= stopwords, lowercase = lowercase)

        df["ref"] = tokenization(df["reference"], tool = tool, language = lang, pos_to_remove = pos_list, ent_to_remove = ent_list, stop_words_to_remove= stopwords, lowercase = lowercase)

    elif lang == "fi":
        word_vect = KeyedVectors.load('fi_vectors.kv')

        df["pt"] = tokenization(df["translation"], tool = tool, language = lang ,lowercase = lowercase)
        df["pt"] = match_regex(df["pt"], letters = let, letters_and_numbers = letandnum)

        df["ref"] = tokenization(df["reference"], tool = tool, language = lang, lowercase = lowercase)
        df["ref"] = match_regex(df["ref"], letters = let, letters_and_numbers = letandnum)

    elif lang == "zh":
        word_vect = KeyedVectors.load('zh_vectors.kv')

        df["pt"] = tokenization(df["translation"], tool = tool, language = lang, stop_words_to_remove= stopwords)

        df["ref"] = tokenization(df["reference"], tool = tool, language = lang, stop_words_to_remove= stopwords)

    else:
        return "ooppsiiee, language not defined"

    for mode in ["precision", "recall"]:
        for i in range(1,5):
            df[str(i)+"gram-"+ mode] = df.apply(lambda x: ngram_calc(x.pt, x.ref, mode=mode, ngram=i), axis =1)

    df['wmdist'] = df.apply(lambda x: word_vect.wmdistance(x["pt"],x["ref"]), axis=1)
    
    df["pt_len"] = df['pt'].str.len()
    df["ref_len"] = df['ref'].str.len()

    df = df.drop(["reference", "translation", "avg-score", "annotators"], axis=1)
    
    return df




In [5]:
en_pairs = ['de-en', 'ru-en', 'zh-en']

fi_df = pd.read_csv("corpus\en-fi\scores.csv")
zh_df = pd.read_csv("corpus\en-zh\scores.csv")
en_df = pd.read_csv("corpus\cs-en\scores.csv")

for pair in en_pairs:
    new_df = pd.read_csv(os.path.join('corpus',pair, 'scores.csv'))
    en_df = en_df.append(new_df, ignore_index=True)


In [6]:
model1 = the_preprocessor2(en_df,"en", tool="spacy", pos_list=['PUNCT','NUM'], ent_list=['PERSON','ORG'], stopwords=False, lowercase=True, let=True, letandnum=False)

NameError: name 'tokenization' is not defined